In [56]:
# importing libraries & const variables
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re

DATE_XPATH = "//time[@class='_1o9PC Nzb55']"
LOCATION_XPATH = "//a[@class='O4GlU']"
DESC_XPATH = '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span'
VIEWS_VIDEO_XPATH = '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/section[2]/div/span'
POPUP_OF_COUNT_OF_LIKES_IF_VIDEO_XPATH = '/html/body/div[1]/section/main/div/div[1]/article/div[3]/section[2]/div/div/div[4]'
LIKES_COUNT_XPATH = "//div[@class='Nm9Fw']"


In [57]:
"""
Post class
#     link
#     likes_count
#     location
#     description
#     hashtags
#     type
#     date
#     likes = []
"""

class Post:

    #TO-DO: comments
    def __init__(self, link):
        self.link = link
        self.likes_count = 0 # In getType method
        self.date = ''
        self.location = ''
        self.description = ''
        self.hashtags = []
        self.type = ''
        self.likes = []
        
    def beginToScape(self):
        driver.get(self.link)
        time.sleep(2)
        self.date = self.getDate()
        self.location = self.getLocation()

        self.description = self.getDescription()
        self.hashtags = self.getHashtags()
        
        self.type = self.getType()
        
#       Get liked accounts (too long to wait for 
#                           scapping & fails in some cases according to your Internet connection)  
        if self.type == 'photo':
            self.likes = self.setLikesAccounts()

    def getLocation(self):
        location = ''
        try:
            location = driver.find_element_by_xpath(f"{LOCATION_XPATH}").text
            location = location.replace(',', ' ')
        except:
            pass
        return location
    
    def getDate(self):
        date = ''
        try:
            date = driver.find_element_by_xpath(f"{DATE_XPATH}").text
            date = date.replace(',', '').replace(' ', '-')
        except:
            pass
        return date
    
    def getDescription(self):
        description = ''
        try:
            description = driver.find_element_by_xpath(f"{DESC_XPATH}").text
        except:
            pass
        return description
        
    def getHashtags(self):
        return [tag.strip("#") for tag in self.description.split() if tag.startswith("#")]
    
    def getType(self):
        typeOfPost = "video"
        try:
            driver.find_element_by_xpath(f"{VIEWS_VIDEO_XPATH}").click()
            likes = driver.find_element_by_xpath(f"{POPUP_OF_COUNT_IF_VIDEO_XPATH}").text
            likes = likes.replace(',', '') # case if 1,909
            self.likes_count = int(re.findall(r'\d+', likes)[0])
        except:
            typeOfPost = 'photo'
            self.likes_count = self.getLikesCount()
        return typeOfPost
    
    def getLikesCount(self):
        count = 0
        likes = driver.find_element_by_xpath(f"{LIKES_COUNT_XPATH}").text
        likes = likes.replace(',', '') # case if 1,909
        if 'Liked by' in likes:
            count += 1
        count += int(re.findall(r'\d+', likes)[0])
        return count
    
    def setLikesAccounts(self):
        likes = []
        # first occuracy of like
        try:
            likes.append(driver.find_element_by_xpath("//a[@class='FPmhX notranslate  cqXBL']").text)
        except:
            pass
        and_other_likes = []
        try:
            and_others = "/html/body/div[1]/section/main/div/div[1]/article/div[3]/section[2]/div/div[2]/button"
            and_other_likes = scrollSpecificElement('likes', None, and_others, "//a[@class='FPmhX notranslate MBL3Z']")
        except:
            pass
        try:
            and_others = "/html/body/div[1]/section/main/div/div[1]/article/div[3]/section[2]/div/div/button"
            and_other_likes_2 = scrollSpecificElement('likes', None, and_others, "//a[@class='FPmhX notranslate MBL3Z']")
            for acc in and_other_likes_2:
                if acc not in and_other_likes:
                    and_other_likes.append(acc)
        except:
            pass
        likes = likes + and_other_likes
        return likes
    
    def __str__(self):
        return f"--Post--{self.link}\nType of post: {self.type}\nLikes count {self.likes_count}\nDescription: {self.description}\nLocation: {self.location}\nDate: {self.date}"
    

In [58]:
"""
InstagramAccount class
#     account_name
#     posts = []
#     followers = []
#     following = []
#     followers_count
#     following_count
#     links
"""

class InstagramAccount:
    def __init__(self, account_name):
        self.account_name = account_name
        self.posts = [] 
        self.followers = []
        self.following = {}
        self.followers_count = 0
        self.following_count = 0
        self.links = [] # in scrollPageTillEnd
        
    def scrapeData(self):
        driver.get(f'https://www.instagram.com/{self.account_name}/')
        time.sleep(3)
#         Scrape accounts of followers and following(too long)
#         self.setFollowers()
#         self.setFollowing()
        self.setFollowersCount()
        self.setFollowingCount()
        time.sleep(1)
        self.scrollPageTillEnd(5)
        self.goThroughPosts()

    # Get all posts links 
    def getLinksOfPosts(self):
        posts = driver.find_elements_by_xpath("//a[contains(@href,'/p/')]")
        links = []
        for post in posts: 
            links.append(post.get_attribute("href"))
        return links


    def scrollPageTillEnd(self, timeout):
        scroll_pause_time = timeout
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            links = self.getLinksOfPosts()
            for link in links:
                if link not in self.links:
                    self.links.append(link)
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(scroll_pause_time)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # If heights are the same it will exit the function
                break
            last_height = new_height
            
    def goThroughPosts(self):
        for link in self.links:
            post = Post(link)
            post.beginToScape()
            print(post)
            self.add_post(post)
            time.sleep(2)
    
    def add_post(self, post):
        self.posts.append(post)
        
    def setFollowers(self):
        followers = scrollSpecificElement(
            'followers',
            partial_link = 'followers'
        )
        self.followers = followers
    
    def setFollowing(self):
        following = scrollSpecificElement(
            'following',
            partial_link = 'following'
        )
        self.following = following
    
    def setFollowingCount(self):
        following = driver.find_element_by_partial_link_text('following').text        
        following = following.replace(',', '') # case if 1,909
        following_count = re.findall(r'\d+', following)[0]
        self.following_count = following_count
        
        
    def setFollowersCount(self):
        followers = driver.find_element_by_partial_link_text('followers').text
        followers = followers.replace(',', '') # case if 1,909
        followers_count = re.findall(r'\d+', followers)[0]
        self.followers_count = followers_count
    
    def __str__(self):
        return f"--------{self.account_name}--------\nHas followers_count -> {self.followers_count}  {len(self.followers)}\nHas following_count -> {self.following_count}  {len(self.following)}\nHas posts count -> {len(self.posts)}\n\n" 

In [59]:
# Login method
def login(driver, username, password):
    driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
    time.sleep(2)
    driver.find_element_by_name("username").send_keys(f"{username}")
    driver.find_element_by_name("password").send_keys(f"{password}") 
    driver.find_element_by_name("password").send_keys(u'\ue007')
    time.sleep(5)
    
# Scrolling the element inside page
def scrollSpecificElement(
    title,
    partial_link = None,
    xpath = None,
    to_append_text_xpath = "//span[@class='Jv7Aj mArmR MqpiF  ']"):

    start = time.time()
    output = []
    
    if partial_link:
        driver.find_element_by_partial_link_text(f"{partial_link}").click()
    elif xpath:
        driver.find_element_by_xpath(f"{xpath}").click()
    time.sleep(2)
    output = scrapScrolling(driver, output, to_append_text_xpath)

    end = time.time()
    print(f"{title} was collected by {end - start}, and collect data of {len(output)}", )
    
    #click X to close
    driver.find_elements_by_xpath(f"//div[@class='WaOAr']")[1].click()
    return output

def scrapScrolling(driver, output, to_append_text_xpath):
    prev = -1
    
    while prev != len(output):
        prev = len(output)
        elements = driver.find_elements_by_xpath(f"{to_append_text_xpath}")
        for element in elements:
            if element.text not in output:
                output.append(element.text)
            
        # Scrolling till the end
        driver.execute_script("arguments[0].scrollIntoView();", elements[-1])
        time.sleep(0.5)
    return output


In [61]:
import Constants # provide ur username and password in Constants.py

driver = webdriver.Firefox()
login(driver, Constants.username, Constants.password)

In [62]:
users = [
    'fariza2912', 'just_4ina'
#     'sapargalinauryzbay', 'justaskhat', 'timur.a.k_03', 'abyllash', '_sanaqov', 'zhyldabyl', 
#     'mahmetsultanov', 'hamitove', 'xsayat', 'ospan_sm', 'temirlanshaikenov', 
#     'kud_adi', 'fearless_white_master', 
#     'aigeriim.s', 'sandu2065mail.ru', 'rtolganay', 'ngm.orz',
#     'perizat.mukhanova', 'ollzhaa',
#     'anarabildina', 'almagambetoff', '_janb0', 'jabrail.chumakov', 
#     'nur.rymtayev', 'sultan_aytzhan', 'yersultanrais'
]
output = []
for user in users:
    account = InstagramAccount(user)
    account.scrapeData()
    output.append(account)
    print("\n\n")

likes was collected by 9.966497898101807, and collect data of 94
--Post--https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/
Type of post: photo
Likes count 95
Description: 
Location: 
Date: NOVEMBER-6
likes was collected by 12.132357597351074, and collect data of 101
--Post--https://www.instagram.com/p/CHI01mFBvWjc7t0Zslpe0-jNmPLZhBQGMCfJQM0/
Type of post: photo
Likes count 102
Description: 
Location: 
Date: NOVEMBER-4
likes was collected by 10.298786878585815, and collect data of 90
--Post--https://www.instagram.com/p/CDjdgJMBl55D5Gx6kRq-NmI9iwqlo5_scthShY0/
Type of post: photo
Likes count 92
Description: Hi
Location: 
Date: AUGUST-6
likes was collected by 12.478109121322632, and collect data of 119
--Post--https://www.instagram.com/p/B9bfTK_hKzWIk7e8LjoacygGKHqY6nklkLJG900/
Type of post: photo
Likes count 119
Description: 
Location: Kaspi Bank (Head Office)
Date: MARCH-7
likes was collected by 12.782769203186035, and collect data of 123
--Post--https://www.instagram

In [63]:
for acc in output:
    print(acc.account_name, len(acc.posts))

fariza2912 15
just_4ina 5


In [65]:
for acc in output:
    for post in acc.posts:
        print(post,'\n')

--Post--https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/
Type of post: photo
Likes count 95
Description: 
Location: 
Date: NOVEMBER-6 

--Post--https://www.instagram.com/p/CHI01mFBvWjc7t0Zslpe0-jNmPLZhBQGMCfJQM0/
Type of post: photo
Likes count 102
Description: 
Location: 
Date: NOVEMBER-4 

--Post--https://www.instagram.com/p/CDjdgJMBl55D5Gx6kRq-NmI9iwqlo5_scthShY0/
Type of post: photo
Likes count 92
Description: Hi
Location: 
Date: AUGUST-6 

--Post--https://www.instagram.com/p/B9bfTK_hKzWIk7e8LjoacygGKHqY6nklkLJG900/
Type of post: photo
Likes count 119
Description: 
Location: Kaspi Bank (Head Office)
Date: MARCH-7 

--Post--https://www.instagram.com/p/B6lkUZfhUWgZ5JqUIrCR0gx_PeJKVC1ZduIy1c0/
Type of post: photo
Likes count 125
Description: 
Location: 
Date: DECEMBER-28-2019 

--Post--https://www.instagram.com/p/B49CNFbAJqJF-Rnbp1eNenkexGguT6xaYgi0i00/
Type of post: photo
Likes count 109
Description: Be loved
Location: 
Date: NOVEMBER-17-2019 

--Post--https://www

In [66]:
output_to_csv = output
# https://realpython.com/python-csv/

# code of csv file

In [67]:
posts_count = 0
for acc in output_to_csv:
    print(acc.account_name)
    posts_count+= (len(acc.posts))
    for post in acc.posts:
#         print(post.date)
        if  post.date.count('-') == 1:
            post.date = (post.date+"-2020")
print(posts_count)

fariza2912
just_4ina
20


CSV file
-
Instructions
1. Collect the parsed data into one place in the form of CSV file.
2. Use semicolon (;) as a separator.
3. Use UTF-8 as an encoding.


###### Things to pay attention

-
1. Make sure that you have at least 1000 (distinct) rows of data in your CSV file.
2. If your parsed data is too big (>million rows of data), include only part of it in your CSV
file.

In [68]:
# https://stackoverflow.com/questions/55928947/how-to-delimit-a-csv-file-with-semicolons
import csv
# 
#         self.account_name = account_name
#         self.posts = [] 
#         self.followers = []
#         self.following = {}
#         self.followers_count = 0
#         self.following_count = 0

with open('users.csv','w',newline='',encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file,delimiter=';')
    csv_writer.writerow(["account_name", "followers_count", "following_count"]) #"followers", "following"])
    
    for user in output_to_csv:
#         followers = " ".join(user.followers)
#         following = " ".join(user.following)
        csv_writer.writerow(
            [f"{user.account_name}", f"{user.followers_count}", 
             f"{user.following_count}"
#              f"{followers}", 
#              f"{following}"
            ])

with open('users.csv','r',newline='',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for row in csv_reader:
        print(row)

['account_name', 'followers_count', 'following_count']
['fariza2912', '1217', '368']
['just_4ina', '295', '297']


In [69]:
#    POSTS
#         self.likes_count = 0
#         self.location = self.getLocation()
#         self.description = self.getDescription()
#         self.hashtags = self.getHashtags()
#         self.typeOfPhoto = self.getType()
#         if self.typeOfPhoto == 'photo':
#             self.likes = self.setLikesAccounts()
import csv

# account	posts	like_count	location	type
# 1	1	10		
# 1	2	11		
# 1	3	13		
# 2	1	11		
# 2	2	1		
# 2	3	22		
# 2	4	33		
# hastags description "likes"
#f"{post.description}", 
#                 f"{post.typeOfPhoto}", f"{hashtags}", f"{likes}"
with open('posts.csv','w',newline='',encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file,delimiter=';')
    
    csv_writer.writerow(
        [f"account", "posts", "likes_count", 
         f"location", f"type", "description", "hashatags", 'date'
        ])
    for user in output:
        for post in user.posts:
#             likes = " ".join(post.likes)
            post.description = post.description.replace(',', ' ')
            hashtags = " ".join(post.hashtags)
            csv_writer.writerow(
                [f"{user.account_name}", f"{post.link}", f"{post.likes_count}", 
                 f"{post.location}", f"{post.type}", f"{post.description}",f"{hashtags}", f"{post.date}"
                ])

with open('posts.csv','r',newline='',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for row in csv_reader:
        print(row)


['account', 'posts', 'likes_count', 'location', 'type', 'description', 'hashatags', 'date']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', '95', '', 'photo', '', '', 'NOVEMBER-6-2020']
['fariza2912', 'https://www.instagram.com/p/CHI01mFBvWjc7t0Zslpe0-jNmPLZhBQGMCfJQM0/', '102', '', 'photo', '', '', 'NOVEMBER-4-2020']
['fariza2912', 'https://www.instagram.com/p/CDjdgJMBl55D5Gx6kRq-NmI9iwqlo5_scthShY0/', '92', '', 'photo', 'Hi', '', 'AUGUST-6-2020']
['fariza2912', 'https://www.instagram.com/p/B9bfTK_hKzWIk7e8LjoacygGKHqY6nklkLJG900/', '119', 'Kaspi Bank (Head Office)', 'photo', '', '', 'MARCH-7-2020']
['fariza2912', 'https://www.instagram.com/p/B6lkUZfhUWgZ5JqUIrCR0gx_PeJKVC1ZduIy1c0/', '125', '', 'photo', '', '', 'DECEMBER-28-2019']
['fariza2912', 'https://www.instagram.com/p/B49CNFbAJqJF-Rnbp1eNenkexGguT6xaYgi0i00/', '109', '', 'photo', 'Be loved', '', 'NOVEMBER-17-2019']
['fariza2912', 'https://www.instagram.com/p/B0AQ7DhBGzdLwfD-BoV99vClHNztofW

In [70]:
with open('post_likes_desc.csv','w',newline='',encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file,delimiter=';')
    
# hastags description "likes"
#f"{post.description}", 
#                 f"{post.typeOfPhoto}", f"{hashtags}", f"{likes}"
    csv_writer.writerow(
        [f"account", "posts", "likes",])
    for user in output_to_csv:
        for post in user.posts:
            for like in post.likes:
                print
                csv_writer.writerow(
                        [f"{user.account_name}", f"{post.link}", f"{like}"])

with open('post_likes_desc.csv','r',newline='',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for row in csv_reader:
        print(row)


['account', 'posts', 'likes']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'gabitovnaaidana']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'manatbekuly_n']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'asylbek.dastanov']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'my.meddb']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'asel_aeka']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'sultanov080']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'aidana_bekenova']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'aziza.dzholdasova']
['fariza2912', 'https://www.instagram.com/p/CHQYUBdBGAxG21RsrQTgPNUSii1JeYNSg4biSI0/', 'akk_3011']
['fariza2912', 'https://www.instagram.c

### Locating by XPath

In [72]:
# URL: https://selenium-python.readthedocs.io/locating-elements.html#locating-by-xpath
# <html>
#  <body>
#   <form id="loginForm">
#    <input name="username" type="text" />
#    <input name="password" type="password" />
#    <input name="continue" type="submit" value="Login" />
#    <input name="continue" type="button" value="Clear" />
#   </form>
# </body>
# <html>

# login_form = driver.find_element_by_xpath("/html/body/form[1]")
# login_form = driver.find_element_by_xpath("//form[1]")
# login_form = driver.find_element_by_xpath("//form[@id='loginForm']")

# username = driver.find_element_by_xpath("//form[input/@name='username']")
# username = driver.find_element_by_xpath("//form[@id='loginForm']/input[1]")
# username = driver.find_element_by_xpath("//input[@name='username']")

### Locating Hyperlinks by Link Text

In [73]:

# <html>
#  <body>
#   <p>Are you sure you want to do this?</p>
#   <a href="continue.html">Continue</a>
#   <a href="cancel.html">Cancel</a>
# </body>
# <html>
# continue_link = driver.find_element_by_link_text('Continue')
# continue_link = driver.find_element_by_partial_link_text('Conti')


In [74]:
# heading1 = driver.find_element_by_tag_name('h1')
# content = driver.find_element_by_class_name('content')
# content = driver.find_element_by_css_selector('p.content')



In [4]:
#Comment add wait condition if it will be needed